In [ ]:
import sys
import warnings
warnings.filterwarnings("ignore")
sys.path.append("../")
from lib.metadata import MetadataListTuep
from lib.signals import EegProcessorTuep
from lib.spectral import FourierAnalysis
from lib.filters import FilterBank
from lib.stats import StatisticalTests
from lib.bss import EogDenoiser, EmgDenoiser
from lib.visuals import plot_eeg_windows, plot_eeg_spectrum

### * Scales the EEG signal
### * Select a subset of the channels

In [ ]:
PATIENT = "aaaaaebo"
FILENAME = "aaaaaebo_s001_t000"
PERIOD = [85, 95]

###########################################################
metadata = MetadataListTuep()
current_metadata = metadata.get(PATIENT, FILENAME)
print(current_metadata)

###########################################################
processor = EegProcessorTuep(current_metadata["full_file"])
processor.scale()
processor.select_channels()

###########################################################
fourier = FourierAnalysis(processor.sampling_frequency)
frequency_range, spectral_components = fourier.run_fast_fourier_transform(processor._data)

###########################################################
plot_eeg_windows(processor._data, current_metadata, processor.selected_channels, processor.sampling_frequency, PERIOD)

###########################################################
plot_eeg_spectrum(frequency_range, spectral_components, processor.selected_channels, "FP1")

### * Resample the EEG to 256Hz

In [ ]:
###########################################################
processor.resample()

###########################################################
fourier = FourierAnalysis(processor.sampling_frequency)

###########################################################
plot_eeg_windows(processor._data, current_metadata, processor.selected_channels, processor.sampling_frequency, PERIOD)


### * Remove the drift noise, high frequency oscillations, power line noise

In [ ]:
###########################################################
processor.filter_bank = FilterBank(0)
processor.remove_drift()
processor.remove_hfo()
processor.remove_power_noise()

###########################################################
fourier = FourierAnalysis(processor.sampling_frequency)
frequency_range, spectral_components = fourier.run_fast_fourier_transform(processor._data)

###########################################################
plot_eeg_windows(processor._data, current_metadata, processor.selected_channels, processor.sampling_frequency, PERIOD)

###########################################################
plot_eeg_spectrum(frequency_range, spectral_components, processor.selected_channels, "FP1")

### * Separated sources using iWASOBI

In [ ]:
eog_denoiser = EogDenoiser(processor.sampling_frequency)
eeg_sources, clean_eog_eeg = eog_denoiser.apply_by_segments(processor._data[:, PERIOD[0]*processor.sampling_frequency:
                                                            PERIOD[1]*processor.sampling_frequency])
print(eog_denoiser)

###########################################################
plot_eeg_windows(eeg_sources, current_metadata, [f"S{idx}" for idx in range(1, eeg_sources.shape[0] + 1)],
                 processor.sampling_frequency, period=[0, 10])

###########################################################
plot_eeg_windows(clean_eog_eeg, current_metadata, processor.selected_channels,
                 processor.sampling_frequency, period=[0, 10])

### * Compute sources using BSS Canonical Correlation Analysis

In [ ]:
emg_denoiser = EmgDenoiser(processor.sampling_frequency)
eeg_sources, clean_emg_eeg = emg_denoiser.apply_by_segments(processor._data[:, PERIOD[0]*processor.sampling_frequency:
                                                            PERIOD[1]*processor.sampling_frequency])
print(emg_denoiser)

###########################################################
plot_eeg_windows(eeg_sources, current_metadata, [f"S{idx}" for idx in range(1, eeg_sources.shape[0] + 1)],
                 processor.sampling_frequency, period=[0, 10])

###########################################################
plot_eeg_windows(clean_emg_eeg, current_metadata, processor.selected_channels,
                 processor.sampling_frequency, period=[0, 10])

### * Stationarity tests (KPPS and ADF)

In [ ]:
stats_gateway = StatisticalTests()
stats_gateway.check_stationarity(processor._data[15, :4000])